In [1]:
import pandas as pd
import warnings
import requests, zipfile, io, os
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
warnings.filterwarnings('ignore')

In [2]:
# download file from url, extract it and store it in the dest_folder
def download_and_extract(url, dest_folder):
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall(dest_folder)

In [3]:
# define folders and urls
healthy_path = './healthy'
healthy_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewHealthy/Signal.zip'
parkinson_path = './parkinson'
parkinson_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewPatients/Signal.zip'

In [4]:
download_and_extract(healthy_url, healthy_path)

In [5]:
download_and_extract(parkinson_url, parkinson_path)

In [6]:
# actual downloaded signals
healthy_signals = f'{healthy_path}/Signal/'
parkinson_signals = f'{parkinson_path}/Signal/'

In [7]:
# define Input and labels (0 = healthy, parkinson = 1)
X = []
y = []
seq = 500

In [8]:
# create dataset with sequence length 500

# healthy signals
for filename in os.listdir(healthy_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(healthy_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(0)

# parkinson signals
for filename in os.listdir(parkinson_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(parkinson_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(1) 


In [9]:
len(X)

29088

In [10]:
# convert lists to np arrays
X = np.array(X)
y = np.array(y)

In [11]:
X.shape

(29088, 500, 6)

In [12]:
y.shape

(29088,)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

In [15]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_valid = scaler.transform(X_valid.reshape(-1, X_valid.shape[-1])).reshape(X_valid.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [16]:
model = keras.models.Sequential([
keras.layers.GRU(128, return_sequences=True,  input_shape=(X_train.shape[1], 6)),
keras.layers.GRU(128),
keras.layers.Dense(1, activation="sigmoid")
])


In [17]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [18]:
checkpoint_filepath = "model.h5"
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.fit(X_train, y_train, epochs = 30, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Epoch 1/30
284/582 [=============>................] - ETA: 50s - loss: 1.0657 - accuracy: 0.6213

In [ ]:
model.fit(X_train, y_train, epochs = 20, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

In [ ]:
model.fit(X_train, y_train, epochs = 20, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
model.evaluate(X_valid, y_valid)


146/146 [==============================] - 2s 14ms/step - loss: 0.3749 - accuracy: 0.9218


[0.37493348121643066, 0.9217877388000488]

In [ ]:
model.evaluate(X_test, y_test)

182/182 [==============================] - 3s 14ms/step - loss: 0.3548 - accuracy: 0.9211


[0.3547653555870056, 0.9211069345474243]

In [ ]:
!ls

healthy  model.h5  parkinson  sample_data


In [ ]:
from google.colab import files
files.download('model.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>